# Languages pivot table

In [1]:
import json
from urllib.request import urlopen
import pandas as pd
import numpy as np

/home/ednilson/.virtualenvs/jupyter/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Get collection information from ArticleMeta 

In [2]:
AMC_URL = "http://articlemeta.scielo.org/api/v1/collection/identifiers/"
amc_data = pd.DataFrame(json.load(urlopen(AMC_URL)))

In [3]:
amc_data.head(6)

,acron,acron2,code,document_count,domain,has_analytics,is_active,journal_count,name,original_name,status,type
0,arg,ar,arg,38295.0,www.scielo.org.ar,True,True,"{'deceased': 22, 'current': 125}","{'en': 'Argentina', 'es': 'Argentina', 'pt': '...",Argentina,certified,journals
1,chl,cl,chl,62746.0,www.scielo.cl,True,True,"{'suspended': 1, 'deceased': 13, 'current': 105}","{'en': 'Chile', 'es': 'Chile', 'pt': 'Chile'}",Chile,certified,journals
2,col,co,col,68855.0,www.scielo.org.co,True,True,"{'suspended': 7, 'current': 226}","{'en': 'Colombia', 'es': 'Colombia', 'pt': 'Co...",Colombia,certified,journals
3,cub,cu,cub,33492.0,scielo.sld.cu,True,True,"{'suspended': 4, 'deceased': 2, 'current': 61}","{'en': 'Cuba', 'es': 'Cuba', 'pt': 'Cuba'}",Cuba,certified,journals
4,esp,es,esp,37696.0,scielo.isciii.es,True,True,"{'suspended': 11, 'deceased': 6, 'current': 43}","{'en': 'Spain', 'es': 'España', 'pt': 'Espanha'}",España,certified,journals
5,mex,mx,mex,61585.0,www.scielo.org.mx,True,True,"{'suspended': 44, 'deceased': 12, 'current': 159}","{'en': 'Mexico', 'es': 'Mexico', 'pt': 'Mexico'}",Mexico,certified,journals


Some collections won't be analyzed, mainly to avoid duplicates
(there are articles in more than one collection).
The `spa` (*Public Health* collection) should have part of it
kept in the result, but it's not a collection
whose journals/articles are assigned to a single country.
The collections below are linked to a single country:

In [4]:
dont_evaluate = ["bio", "cci", "cic", "ecu", "psi", "pry", "rve", "rvo", "rvt", "sss", "spa", "wid"]
amc_names_map = {
    "code": "collection",
    "acron2": "origin",
}
amc_pairs = amc_data \
    [(amc_data["acron2"].str.len() == 2) &
     ~amc_data["code"].isin(dont_evaluate)] \
    [list(amc_names_map.keys())] \
    .rename(columns=amc_names_map) \
    .assign(origin=lambda df: df["origin"].str.upper())
amc_pairs

,collection,origin
0,arg,AR
1,chl,CL
2,col,CO
3,cub,CU
4,esp,ES
5,mex,MX
6,prt,PT
8,scl,BR
11,sza,ZA
12,ven,VE


## ISSN selection from `spa`

These journals in the `spa` collection have the following countries:

In [5]:
spa_issn_country = pd.DataFrame([
    ("0021-2571", "IT"),
    ("0042-9686", "CH"),
    ("1020-4989", "US"),
    ("1555-7960", "US"),
], columns=["issn", "origin"])
spa_issn_country # For collection = "spa", only!

,issn,origin
0,0021-2571,IT
1,0042-9686,CH
2,1020-4989,US
3,1555-7960,US


## Languages dataset

This dataset is the
[Network spreadsheet/CSV pack](https://static.scielo.org/tabs/tabs_network.zip)
 which can be found in the
[SciELO Analytics report](https://analytics.scielo.org/w/reports)
web page.
The first two rows of it are:

#### Unzip the CSV file

In [6]:
import zipfile
# Use the Zip file in jcatalog/data/scielo
with zipfile.ZipFile('../../data/scielo/tabs_network_181210.zip', 'r') as zip_ref:
    zip_ref.extract('documents_languages.csv', 'csv_files')

In [7]:
dataset = pd.read_csv("csv_files/documents_languages.csv", keep_default_na=False)
dataset.head(3).T

,0,1,2
extraction date,2018-12-10,2018-12-10,2018-12-10
study unit,document,document,document
collection,scl,scl,scl
ISSN SciELO,0100-879X,0100-879X,0100-879X
ISSN's,0100-879X;1414-431X,0100-879X;1414-431X,0100-879X;1414-431X
title at SciELO,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...
title thematic areas,Biological Sciences;Health Sciences,Biological Sciences;Health Sciences,Biological Sciences;Health Sciences
title is agricultural sciences,0,0,0
title is applied social sciences,0,0,0
title is biological sciences,1,1,1


#### Simplify the column names

In [8]:
names_map = {
    "ISSN SciELO": "issn",
    "title at SciELO":"title",
    "document publishing ID (PID SciELO)": "pid",
    "document type":"type",
    "document languages": "languages",
    "document is citable": "is_citable",
    "document publishing year": "year",
    "document pt": "document_pt",
    "document es": "document_es",
    "document en": "document_en",
    "document other languages": "document_other_languages"
}
df = dataset[list(names_map.keys())].rename(columns=names_map)
df[:5].T

,0,1,2,3,4
issn,0100-879X,0100-879X,0100-879X,0100-879X,0100-879X
title,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...
pid,S0100-879X1998000800006,S0100-879X1998000800011,S0100-879X1998000800005,S0100-879X1998000800009,S0100-879X1998000800010
type,research-article,rapid-communication,research-article,rapid-communication,rapid-communication
languages,en,en,en,en,en
is_citable,1,1,1,1,1
year,1998,1998,1998,1998,1998
document_pt,0,0,0,0,0
document_es,0,0,0,0,0
document_en,1,1,1,1,1


#### Add pub_year (ate_1996)

In [9]:
df["pub_year"] = np.where(df['year'] <= 1996, 'ate_1996', df["year"])

#### Add review type

In [10]:
df["tipo_review"] = np.where(df['type'] == "review-article", 1, 0)

#### Add citable_(language)

In [11]:
df["citable_pt"] = np.where((df['document_pt'] == 1) & (df['is_citable'] == 1), 1, 0)
df["citable_es"] = np.where((df['document_es'] == 1) & (df['is_citable'] == 1), 1, 0)
df["citable_en"] = np.where((df['document_en'] == 1) & (df['is_citable'] == 1), 1, 0)
df["citable_other_lang"] = np.where((df['document_other_languages'] == 1) & (df['is_citable'] == 1), 1, 0)

In [12]:
df['sum_to_2_more_lang'] = np.sum([df['document_en'], df['document_pt'], df['document_es'], df['document_other_languages']], axis=0)
df[(df['sum_to_2_more_lang'] == 3)].T

,31760,32728,34448,36347,37268,37270,37272,39283,42191,42453,...,808244,808476,808491,808785,808878,809050,809551,809557,809676,809781
issn,0104-1169,0104-1169,0104-1169,0104-1169,0104-1169,0104-1169,0104-1169,0104-1169,0104-1169,0104-1169,...,0102-311X,0102-311X,0102-311X,0102-311X,0102-311X,0102-311X,0104-1290,0102-311X,0102-311X,0102-311X
title,Revista Latino-Americana de Enfermagem,Revista Latino-Americana de Enfermagem,Revista Latino-Americana de Enfermagem,Revista Latino-Americana de Enfermagem,Revista Latino-Americana de Enfermagem,Revista Latino-Americana de Enfermagem,Revista Latino-Americana de Enfermagem,Revista Latino-Americana de Enfermagem,Revista Latino-Americana de Enfermagem,Revista Latino-Americana de Enfermagem,...,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Saúde e Sociedade,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Cadernos de Saúde Pública
pid,S0104-11692003000400001,S0104-11692003000600001,S0104-11692004000200001,S0104-11692003000500001,S0104-11692004000700001,S0104-11692004000700002,S0104-11692004000700003,S0104-11692004000400001,S0104-11692004000500001,S0104-11692004000600001,...,S0102-311X2018000300101,S0102-311X2014001300001,S0102-311X2018000700201,S0102-311X2018000800201,S0102-311X2018000700101,S0102-311X2018000900101,S0104-12902018000300682,S0102-311X2018001100101,S0102-311X2018001200201,S0102-311X2018001000101
type,editorial,editorial,editorial,editorial,editorial,research-article,research-article,editorial,editorial,editorial,...,editorial,editorial,editorial,editorial,editorial,editorial,research-article,editorial,editorial,editorial
languages,en;pt;es,en;pt;es,en;es;pt,en;pt;es,en;pt;es,en;pt;es,en;pt;es,en;pt;es,en;es;pt,en;es;pt,...,en;es;pt,en;es;pt,en;pt;es,en;pt;es,en;es;pt,en;es;pt,fr;en;pt,en;es;pt,en;es;pt,en;es;pt
is_citable,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
year,2003,2003,2004,2003,2004,2004,2004,2004,2004,2004,...,2018,2014,2018,2018,2018,2018,2018,2018,2018,2018
document_pt,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
document_es,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,1,1,1
document_en,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [13]:
df["doc_2_more_lang"] = np.where(((df['sum_to_2_more_lang']) >= 2), 1, 0)
df["citable_doc_2_more_lang"] = np.where((df['sum_to_2_more_lang'] >= 2) & (df['is_citable'] == 1), 1, 0)
# remove sum_to_2_more_lang column
del df['sum_to_2_more_lang']

df[(df["doc_2_more_lang"] == 1)].T

,11706,11710,11711,11713,11718,11721,11725,31273,31274,31275,...,838485,839520,839522,839524,839526,839528,839529,839530,839531,839532
issn,0102-311X,0102-311X,0102-311X,0102-311X,0102-311X,0102-311X,0102-311X,0066-782X,0066-782X,0066-782X,...,1688-9339,1688-9339,1688-9339,1688-9339,1688-9339,1688-9339,1688-9339,1688-9339,1688-9339,1688-9339
title,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Arquivos Brasileiros de Cardiologia,Arquivos Brasileiros de Cardiologia,Arquivos Brasileiros de Cardiologia,...,Odontoestomatología,Odontoestomatología,Odontoestomatología,Odontoestomatología,Odontoestomatología,Odontoestomatología,Odontoestomatología,Odontoestomatología,Odontoestomatología,Odontoestomatología
pid,S0102-311X1998000200011,S0102-311X1998000200015,S0102-311X1998000200016,S0102-311X1998000200018,S0102-311X1998000200024,S0102-311X1998000200003,S0102-311X1998000200007,S0066-782X2003001400001,S0066-782X2003001400004,S0066-782X2003001400005,...,S1688-93392015000100001,S1688-93392015000200003,S1688-93392015000200002,S1688-93392015000200006,S1688-93392015000200009,S1688-93392015000200007,S1688-93392015000200005,S1688-93392015000200001,S1688-93392015000200004,S1688-93392015000200008
type,research-article,research-article,research-article,brief-report,article-commentary,research-article,research-article,research-article,research-article,research-article,...,research-article,research-article,research-article,research-article,letter,case-report,research-article,editorial,research-article,case-report
languages,en;pt,es;pt,pt;es,en;pt,es;pt,en;pt,pt;es,en;pt,en;pt,en;pt,...,en;es,en;es,en;es,en;es,en;es,en;es,en;es,en;es,en;es,en;es
is_citable,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,1,0,1,1
year,1998,1998,1998,1998,1998,1998,1998,2003,2003,2003,...,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015
document_pt,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
document_es,0,1,1,0,1,0,1,0,0,0,...,1,1,1,1,1,1,1,1,1,1
document_en,1,0,0,1,0,1,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [15]:
values_list = [
    "pid",
    "is_citable", 
    "tipo_review",
    "document_pt", 
    "document_es", 
    "document_en", 
    "tipo_review",
    "citable_pt",
    "citable_es",
    "citable_en",
    "citable_other_lang",
    "citable_doc_2_more_lang"]

td = df.pivot_table(
     index=["issn"],
     values=values_list,
     columns=["pub_year"],
     aggfunc=np.count_nonzero,
     fill_value=0)
td[:11].T

issn                              0001-3714  0001-3765  0001-6002  0001-6365  \
                        pub_year                                               
citable_doc_2_more_lang 1997              0          0          0          0   
                        1998              0          0          0          0   
                        1999              0          0          0          0   
                        2000              0          0          0          0   
                        2001              0          0          0          0   
                        2002              0          0          0          0   
                        2003              0          0          0          0   
                        2004              0          0          0          0   
                        2005              0          0          0          0   
                        2006              0          0          0          0   
                        2007              0          0          0          0   
                        2008              0          0          0          0   
                        2009              0          0          0          0   
                        2010              0          0          0          0   
                        2011              0          0          7          0   
                        2012              0          0         26          0   
                        2013              0          0         14          0   
                        2014              0          0          0          0   
                        2015              0          0          0          0   
                        2016              0          0          0          0   
                        2017              0          0          0          0   
                        2018              0          0          0          0   
                        2019              0          0          0          0   
                        ate_1996          0          0          0          0   
citable_en              1997              0          0          0          0   
                        1998             33          0          0          0   
                        1999             64          0          0          0   
                        2000              0         64          0          0   
                        2001              0         74          0          0   
                        2002              0         51          0          0   
...                                     ...        ...        ...        ...   
is_citable              2015              0        185         36          0   
                        2016              0        200         29          0   
                        2017              0        286         25          0   
                        2018              0        278         17          0   
                        2019              0          0          0          0   
                        ate_1996          0          0          0          0   
pid                     1997              0          0          0          0   
                        1998             33          0          0          0   
                        1999             64          0          0         41   
                        2000              0        119         31         50   
                        2001              0         74         35         48   
                        2002              0        104         31         60   
                        2003              0         41         41         51   
                        2004              0         85         48         49   
                        2005              0         52         44         57   
                        2006              0         65         48         74   
                        2007              0         62         58        114   
